In [12]:
import pandas as pd
import os

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df.head()

,Days_Since_Start,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Volume,TSLA_RSI_7,TSLA_RSI_14,TSLA_MACD,TSLA_MACD_Signal,...,NEE_MACD_Signal,HD_Open,HD_High,HD_Low,HD_Close,HD_Volume,HD_RSI_7,HD_RSI_14,HD_MACD,HD_MACD_Signal
0,196,1.424667,1.483333,1.394667,1.460667,37297500.0,92.220196,43.408794,-0.064685,-0.083391,...,0.059857,19.278965,19.349948,19.058918,19.215080,17061600,25.229445,25.954283,-0.802271,-0.824908
1,197,1.456667,1.456667,1.336667,1.353333,27379500.0,71.260949,39.206237,-0.058209,-0.078355,...,0.071355,18.987938,19.896520,18.895661,19.825537,23279400,44.368562,43.190715,-0.756203,-0.811167
2,198,1.377333,1.393333,1.300000,1.348000,18787500.0,74.198423,39.159154,-0.052897,-0.073263,...,0.079636,19.719066,19.882326,19.413839,19.499018,17161300,37.425292,44.047680,-0.737539,-0.796441
3,199,1.366667,1.416667,1.358000,1.400000,14367000.0,72.916658,46.915168,-0.043984,-0.067407,...,0.087182,19.669378,20.208849,19.669378,20.031391,26142700,44.414890,52.659520,-0.672043,-0.771562
4,200,1.412667,1.437333,1.404000,1.419333,9804000.0,65.010249,57.983173,-0.034957,-0.060917,...,0.093594,19.974605,20.130768,19.804246,20.052687,18979200,49.562788,54.446405,-0.611372,-0.739524


In [13]:
import pandas as pd

file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Days_Since_Start'])
df.set_index('Days_Since_Start', inplace=True)

# 确保其他列是数值类型，去掉可能的字符串类型列（例如日期）
df = df.apply(pd.to_numeric, errors='coerce')

# 检查数据的头部
print(df.head())

                  TSLA_Open  TSLA_High  TSLA_Low  TSLA_Close  TSLA_Volume  \
Days_Since_Start                                                            
196                1.424667   1.483333  1.394667    1.460667   37297500.0   
197                1.456667   1.456667  1.336667    1.353333   27379500.0   
198                1.377333   1.393333  1.300000    1.348000   18787500.0   
199                1.366667   1.416667  1.358000    1.400000   14367000.0   
200                1.412667   1.437333  1.404000    1.419333    9804000.0   

                  TSLA_RSI_7  TSLA_RSI_14  TSLA_MACD  TSLA_MACD_Signal  \
Days_Since_Start                                                         
196                92.220196    43.408794  -0.064685         -0.083391   
197                71.260949    39.206237  -0.058209         -0.078355   
198                74.198423    39.159154  -0.052897         -0.073263   
199                72.916658    46.915168  -0.043984         -0.067407   
200             

In [14]:
rows, cols = df.shape
print(f"DataFrame 共有 {rows} 行, {cols} 列")


DataFrame 共有 2633 行, 91 列


In [15]:
# 检查数据的列名
print(df.columns)

# 检查是否有缺失值
print(df.isnull().sum())


Index(['TSLA_Open', 'TSLA_High', 'TSLA_Low', 'TSLA_Close', 'TSLA_Volume',
       'TSLA_RSI_7', 'TSLA_RSI_14', 'TSLA_MACD', 'TSLA_MACD_Signal',
       'AAPL_Open', 'AAPL_High', 'AAPL_Low', 'AAPL_Close', 'AAPL_Volume',
       'AAPL_RSI_7', 'AAPL_RSI_14', 'AAPL_MACD', 'AAPL_MACD_Signal', 'GE_Open',
       'GE_High', 'GE_Low', 'GE_Close', 'GE_Volume', 'GE_RSI_7', 'GE_RSI_14',
       'GE_MACD', 'GE_MACD_Signal', 'QQQ_Open', 'QQQ_High', 'QQQ_Low',
       'QQQ_Close', 'QQQ_Volume', 'QQQ_RSI_7', 'QQQ_RSI_14', 'QQQ_MACD',
       'QQQ_MACD_Signal', 'NVDA_Open', 'NVDA_High', 'NVDA_Low', 'NVDA_Close',
       'NVDA_Volume', 'NVDA_RSI_7', 'NVDA_RSI_14', 'NVDA_MACD',
       'NVDA_MACD_Signal', 'UNH_Open', 'UNH_High', 'UNH_Low', 'UNH_Close',
       'UNH_Volume', 'UNH_RSI_7', 'UNH_RSI_14', 'UNH_MACD', 'UNH_MACD_Signal',
       'CAT_Open', 'CAT_High', 'CAT_Low', 'CAT_Close', 'CAT_Volume',
       'CAT_RSI_7', 'CAT_RSI_14', 'CAT_MACD', 'CAT_MACD_Signal', 'AMZN_Open',
       'AMZN_High', 'AMZN_Low', 'AMZN_

In [16]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd

class TradingEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}
    
    def __init__(self, df, window_size=10, initial_balance=10000, max_drawdown=0.3, risk_free_rate=0.01, transaction_cost=0.0005):
        super(TradingEnv, self).__init__()
        
        self.df = df
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.max_drawdown = max_drawdown
        self.risk_free_rate = risk_free_rate
        self.transaction_cost = transaction_cost  # 交易成本降低到 0.05%
        
        # 获取股票代码列表，排除 "Days_Since_Start"
        self.tickers = sorted(set(col.split('_')[0] for col in df.columns if '_' in col and not col.startswith("Days")))
        self.num_stocks = len(self.tickers)
        self.features_per_stock = 9  # 9个特征 (Open, High, Low, Close, Volume, RSI_7, RSI_14, MACD, MACD_Signal)
        
        # 定义状态空间 (window_size 天 * 10 只股票 * 9 特征)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(window_size * 90,), dtype=np.float32)

        # 动作空间 (-0.5 ~ 0.5 代表买入 / 卖出比例，减少剧烈变动)
        self.action_space = spaces.Box(low=-0.5, high=0.5, shape=(self.num_stocks,), dtype=np.float32)

        self.reset()
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.shares_held = np.zeros(self.num_stocks)  # 每只股票的持仓
        self.total_profit = 0
        self.peak_value = self.initial_balance
        self.done = False
        self.history = [self.initial_balance]  # 记录每一步的资产值

        return self._next_observation(), {}
    
    def _next_observation(self):
        feature_cols = [col for col in self.df.columns if col != "Days_Since_Start"]
        obs = self.df.iloc[self.current_step - self.window_size:self.current_step][feature_cols].values.flatten()
        return obs

    def step(self, action):
        if self.done:
            return self._next_observation(), 0, True, False, {}

        close_prices = self.df.iloc[self.current_step, [self.df.columns.get_loc(f"{t}_Close") for t in self.tickers]]
        
        # 计算当前总资产
        total_value = self.balance + np.dot(self.shares_held, close_prices.values)
        
        # 计算目标持仓
        target_values = np.clip(action, -0.5, 0.5) * total_value  # 限制变动幅度
        target_shares = target_values / (close_prices.values + 1e-6)  # 避免除零错误
        delta_shares = target_shares - self.shares_held

        # 执行交易
        for i in range(self.num_stocks):
            current_price = close_prices.iloc[i]
            cost = abs(delta_shares[i]) * current_price * self.transaction_cost  

            if delta_shares[i] > 0:  # 买入
                max_shares = self.balance // (current_price + cost)
                buy_shares = min(max_shares, delta_shares[i])  
                self.shares_held[i] += buy_shares
                self.balance -= buy_shares * current_price + cost

            elif delta_shares[i] < 0:  # 卖出
                sell_shares = min(abs(delta_shares[i]), self.shares_held[i])  
                self.shares_held[i] -= sell_shares
                self.balance += sell_shares * current_price - cost

        # 计算新资产总值
        total_value = self.balance + np.dot(self.shares_held, close_prices.values)
        prev_total_value = self.history[-1] if len(self.history) > 0 else self.initial_balance
        daily_return = (total_value - prev_total_value) / (prev_total_value + 1e-6)

        self.total_profit = total_value - self.initial_balance

        # 计算回撤
        self.peak_value = max(self.peak_value, total_value)
        drawdown = (self.peak_value - total_value) / (self.peak_value + 1e-6)

        # 计算夏普比率
        self.history.append(total_value)
        sharpe_ratio = 0  # 默认值

        if len(self.history) > 30:
            returns = np.diff(self.history[-30:]) / (np.array(self.history[-30:-1]) + 1e-6)
            if len(returns) > 1 and np.std(returns) > 0:
                sharpe_ratio = (np.mean(returns) - self.risk_free_rate) / (np.std(returns) + 1e-6)

        # 计算奖励
        reward = daily_return * 100 - drawdown * 2 + sharpe_ratio * 0.2  # 调整奖励函数
        
        # 如果回撤超过 30%，给予额外惩罚但不终止
        if drawdown >= self.max_drawdown:
            reward -= 10

        # 更新状态
        self.current_step += 1
        self.done = self.current_step >= len(self.df) - 1  # 只在数据用完时终止

        return self._next_observation(), reward, self.done, False, {}
    
    def render(self, mode="human"):
        print(f'Step: {self.current_step}, Balance: {self.balance:.2f}, Shares: {self.shares_held}, Profit: {self.total_profit:.2f}')

print("TradingEnv ready!")


TradingEnv ready!


In [17]:
import gymnasium as gym
import numpy as np

# 假设 df 是你已经加载并预处理好的 DataFrame，包含所有需要的特征
# df = pd.read_csv('your_stock_data.csv')

# 创建环境实例
env = TradingEnv(df)  # 使用你的数据

# 测试环境初始化
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

# 测试 reset 和 _next_observation
obs, info = env.reset()
print("Initial Observation:", obs)


Observation Space: Box(-inf, inf, (900,), float32)
Action Space: Box(-0.5, 0.5, (10,), float32)
Initial Observation: [ 1.42466700e+00  1.48333299e+00  1.39466703e+00  1.46066701e+00
  3.72975000e+07  9.22201958e+01  4.34087939e+01 -6.46850475e-02
 -8.33913485e-02  7.51978817e+00  7.51978817e+00  7.21042614e+00
  7.39038515e+00  1.02447800e+09  1.41132493e+01  2.77689318e+01
 -6.74227941e-02 -2.62855064e-02  5.17271744e+01  5.20804997e+01
  5.11265131e+01  5.16565094e+01  1.02396750e+07  4.15999081e+01
  4.42771895e+01 -9.42785420e-01 -1.24506051e+00  3.90346988e+01
  3.93331427e+01  3.87099192e+01  3.92541428e+01  8.18008000e+07
  5.85526974e+01  4.73683933e+01 -1.46861507e-01 -2.96142639e-01
  2.33627513e-01  2.40734902e-01  2.32022593e-01  2.39817828e-01
  7.98440000e+08  5.35545959e+01  4.24170542e+01 -1.02271605e-02
 -1.21284987e-02  2.44886278e+01  2.48742751e+01  2.43359767e+01
  2.47617950e+01  8.57550000e+06  7.56758722e+01  6.51206712e+01
  9.71865263e-02 -5.71588427e-02  4.45

In [18]:
# 测试执行一步操作
action = np.random.uniform(-1, 1, size=(env.num_stocks,))  # 随机生成动作
print("Test Action:", action)

# 进行一步操作并获取返回的观察值、奖励等
obs, reward, done, truncated, info = env.step(action)
print("New Observation:", obs)
print("Reward:", reward)
print("Done:", done)


Test Action: [-0.7372015  -0.11320734 -0.41322158  0.17686547 -0.30276099 -0.28508757
  0.0907794   0.37716313  0.64954283 -0.22330129]
New Observation: [ 1.45666695e+00  1.45666695e+00  1.33666694e+00  1.35333300e+00
  2.73795000e+07  7.12609494e+01  3.92062370e+01 -5.82086295e-02
 -7.83548047e-02  7.30973612e+00  7.61067180e+00  7.22276563e+00
  7.58027697e+00  1.07495080e+09  3.25979834e+01  4.52739356e+01
 -6.54673241e-02 -3.41218700e-02  5.07378612e+01  5.30344895e+01
  5.07378612e+01  5.27871590e+01  1.40386600e+07  4.96553316e+01
  5.73718569e+01 -8.47155793e-01 -1.16547957e+00  3.86221417e+01
  3.97281380e+01  3.84992537e+01  3.97281380e+01  8.90612000e+07
  6.01264421e+01  6.52173061e+01 -9.48661735e-02 -2.55887346e-01
  2.33627499e-01  2.47154484e-01  2.31564043e-01  2.45778874e-01
  1.16098800e+09  5.79399830e+01  5.31089041e+01 -9.26364943e-03
 -1.15555288e-02  2.51876161e+01  2.53000962e+01  2.42958053e+01
  2.47698307e+01  1.34939000e+07  7.21651289e+01  7.27274288e+01
  

In [19]:
# 测试多步操作
for i in range(10):
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Step {i+1}, Reward: {reward}, Total Profit: {env.total_profit}, Done: {done}")
    if done:
        break


Step 1, Reward: 0.024024798369214524, Total Profit: -12.26803689260305, Done: False
Step 2, Reward: 0.5603837660550204, Total Profit: 43.701591631343035, Done: False
Step 3, Reward: -0.10994254744399426, Total Profit: 32.87580595623331, Done: False
Step 4, Reward: -0.4995076600947365, Total Profit: -16.045522926342528, Done: False
Step 5, Reward: 0.07172229528648782, Total Profit: -7.8597100500701345, Done: False
Step 6, Reward: -0.7849123066329269, Total Profit: -83.75323857047988, Done: False
Step 7, Reward: -3.672087610813569, Total Profit: -438.36747542612284, Done: False
Step 8, Reward: -0.24873465536333214, Total Profit: -452.69907221620815, Done: False
Step 9, Reward: -1.1283154741751238, Total Profit: -549.1517045570763, Done: False
Step 10, Reward: -0.041371295475931144, Total Profit: -542.0383380945041, Done: False


In [20]:
# 测试环境是否能正常结束
env.reset()
while not env.done:
    action = np.random.uniform(-1, 1, size=(env.num_stocks,))
    obs, reward, done, truncated, info = env.step(action)
    print(f"Balance: {env.balance}, Total Profit: {env.total_profit}")


Balance: 2414.671879511305, Total Profit: -19.76884831548341
Balance: 1260.904861094411, Total Profit: -75.67098451681886
Balance: 7302.6609085618475, Total Profit: 40.043819051883474
Balance: 68.09380787859868, Total Profit: -30.12562847079971
Balance: 25.86457973349869, Total Profit: -105.21717167335191
Balance: 13.950543325543777, Total Profit: -92.57621660478435
Balance: 3992.9492982783763, Total Profit: -144.74003894159796
Balance: 23.262148968518986, Total Profit: -406.82473295858836
Balance: 81.85176341963859, Total Profit: -489.18758828209866
Balance: 2762.078515056201, Total Profit: -416.1807148045955
Balance: 344.8073698630478, Total Profit: -415.6305162993158
Balance: 4085.9662361715527, Total Profit: -309.5077782823755
Balance: 2535.4423453603854, Total Profit: -319.3319447532813
Balance: 2337.4234522708093, Total Profit: -435.843498081691
Balance: 4047.332837591794, Total Profit: -470.4392926417495
Balance: 276.2243609102793, Total Profit: -554.8787224606476
Balance: 50.01

In [23]:
import torch
import torch.nn as nn
import gymnasium as gym
import numpy as np
import pandas as pd
from stable_baselines3 import SAC
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

# 自定义 LSTM+Attention 特征提取器
class LSTMAttentionFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, lstm_hidden_size=64, attention_size=32):
        super(LSTMAttentionFeatureExtractor, self).__init__(observation_space, features_dim=lstm_hidden_size)
        
        self.input_dim = observation_space.shape[0]  # shape=(window_size * num_features,)
        self.window_size = 10  # 在 TradingEnv 中设置的窗口大小
        self.num_features = self.input_dim // self.window_size  # 每个时间步的特征数
        self.lstm_hidden_size = lstm_hidden_size
        self.attention_size = attention_size

        # LSTM 层
        self.lstm = nn.LSTM(input_size=self.num_features, hidden_size=self.lstm_hidden_size, batch_first=True)

        # 注意力层
        self.attention = nn.Linear(self.lstm_hidden_size, self.attention_size)
        self.attn_weights = nn.Linear(self.attention_size, 1, bias=False)

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        batch_size = observations.shape[0]
        # 调整输入形状 (batch, window_size, num_features)
        lstm_input = observations.view(batch_size, self.window_size, self.num_features)
        # LSTM 前向传播
        lstm_out, _ = self.lstm(lstm_input)
        # 注意力计算
        attn_scores = torch.tanh(self.attention(lstm_out))  
        attn_weights = torch.softmax(self.attn_weights(attn_scores), dim=1)  
        context_vector = torch.sum(attn_weights * lstm_out, dim=1)  
        return context_vector

# 定义调度函数
def lr_schedule(progress_remaining):
    """
    学习率调度函数
    progress_remaining: 1.0 (开始) 到 0.0 (结束)
    从 0.0003 线性衰减到 0.0001
    """
    return 0.0001 + 0.0002 * progress_remaining

def ent_coef_schedule(progress_remaining):
    """
    熵系数调度函数
    progress_remaining: 1.0 到 0.0
    从 0.1 线性衰减到 0.01
    """
    return 0.01 + 0.09 * progress_remaining

# 读取数据
file_path = "/home/jesse/Projects/RL_Testing/LSTM_Attention/combined_10_stocks_data.csv"
df = pd.read_csv(file_path)

# 创建环境（假设 TradingEnv 已经定义好）
env = TradingEnv(df)
env = Monitor(env)  # 使用 Monitor 包装
env = DummyVecEnv([lambda: env])

# 定义自定义 LSTM+Attention 特征提取器的 SAC policy
policy_kwargs = dict(
    features_extractor_class=LSTMAttentionFeatureExtractor,
    features_extractor_kwargs=dict(lstm_hidden_size=64, attention_size=32),
    net_arch=[256, 256]  # 隐藏层大小
)

# 创建 SAC 模型，同时传入学习率和熵系数调度函数
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    policy_kwargs=policy_kwargs,
    buffer_size=200000,
    learning_rate=lr_schedule,  # 学习率使用调度函数
    batch_size=128,
    gamma=0.99,
    tau=0.005,
    ent_coef="auto",  # 使用自动调节熵系数模式
    target_entropy=-np.prod(env.action_space.shape).astype(np.float32),  # 设定目标熵，可根据实际情况调整
    train_freq=64,
    gradient_steps=64
)


# 训练模型
model.learn(total_timesteps=50000)  # 可根据需要增加训练步数

# 保存模型
model.save("sac_lstm_attention_trained_model")

# 评估模型
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")



Using cuda device
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2.62e+03  |
|    ep_rew_mean     | -2.59e+04 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 71        |
|    time_elapsed    | 147       |
|    total_timesteps | 10488     |
| train/             |           |
|    actor_loss      | 196       |
|    critic_loss     | 214       |
|    ent_coef        | 0.126     |
|    ent_coef_loss   | 5.41      |
|    learning_rate   | 0.000258  |
|    n_updates       | 10368     |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 2.62e+03  |
|    ep_rew_mean     | -2.32e+04 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 69        |
|    time_elapsed    | 302       |
|    total_timesteps | 20976     |
| train/             |           |
|    actor_loss      | 328       |
| 